In [ ]:
import pandas as pd
import json
import os
import pathlib
import peakutils
from BaselineRemoval import BaselineRemoval
import numpy as np
from scipy import signal
from scipy.io import loadmat, savemat
from sklearn.model_selection import KFold, train_test_split
import glob
import matplotlib.pyplot as plt
from scipy import signal
import datetime
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(20,5)

Labelling EEG

In [ ]:
def seperate_EEG(marker,EEG_signal):
    neutral=marker[marker.event==101]
    fear=marker[marker.event==102]
    sad=marker[marker.event==103]
    happy=marker[marker.event==104]
    anger=marker[marker.event==105]
    disgust=marker[marker.event==106]
    start_neutral=int(neutral['latency'])
    end_neutral=int(marker.loc[neutral.index+1,'latency'])
    start_fear=int(fear['latency'])
    end_fear=int(marker.loc[fear.index+1,'latency'])
    start_sad=int(sad['latency'])
    end_sad=int(marker.loc[sad.index+1,'latency'])
    start_happy=int(happy['latency'])
    end_happy=int(marker.loc[happy.index+1,'latency'])
    start_anger=int(anger['latency'])
    end_anger=int(marker.loc[anger.index+1,'latency'])
    start_disgust=int(disgust['latency'])
    end_disgust=int(marker.loc[disgust.index+1,'latency'])
    n_EEG=EEG_signal[:,start_neutral:end_neutral]
    f_EEG=EEG_signal[:,start_fear:end_fear]
    s_EEG=EEG_signal[:,start_sad:end_sad]
    h_EEG=EEG_signal[:,start_happy:end_happy]
    an_EEG=EEG_signal[:,start_anger:end_anger]
    d_EEG=EEG_signal[:,start_disgust:end_disgust]
    return n_EEG,f_EEG,s_EEG,h_EEG,an_EEG,d_EEG
def joining(list):
    # Join based on the '-' delimiter
    pp = ''.join(str(e) for e in list)
    return pp


In [ ]:
files=glob.glob(f"all_data\combined_data\EEG_new1\*.mat")
for file in files:
    EEG=loadmat(file)
    marker=pd.DataFrame(EEG['Event_marker'],columns=['event','latency','urevent'])
    EEG_signal=EEG['eeg_data']
    n_EEG,f_EEG,s_EEG,h_EEG,an_EEG,d_EEG=seperate_EEG(marker,EEG_signal)
    EEG_labelled={'neutral':n_EEG,'fear':f_EEG,'sad':s_EEG,'happy':h_EEG,'anger':an_EEG,'disgust':d_EEG}
    savemat(r'E:\multimodal_signal\all_data\combined_data\EEG_labeled\{}'.format(file[-7:]),EEG_labelled)


Labelling ECG

In [ ]:
def seperate_ECG(start_pt_ecg,Fs_ECG,Fs_EEG,ECG_signal,marker):
    neutral=marker[marker.event==101]
    fear=marker[marker.event==102]
    sad=marker[marker.event==103]
    happy=marker[marker.event==104]
    anger=marker[marker.event==105]
    disgust=marker[marker.event==106]
    start_neutral=int(start_pt_ecg+int(neutral['latency'])*Fs_ECG/Fs_EEG)
    end_neutral=int(start_pt_ecg+int(marker.loc[neutral.index+1,'latency'])*Fs_ECG/Fs_EEG)
    start_fear=int(start_pt_ecg+int(fear['latency'])*Fs_ECG/Fs_EEG)
    end_fear=int(start_pt_ecg+int(marker.loc[fear.index+1,'latency'])*Fs_ECG/Fs_EEG)
    start_sad=int(start_pt_ecg+int(sad['latency'])*Fs_ECG/Fs_EEG)
    end_sad=int(start_pt_ecg+int(marker.loc[sad.index+1,'latency'])*Fs_ECG/Fs_EEG)
    start_happy=int(start_pt_ecg+int(happy['latency'])*Fs_ECG/Fs_EEG)
    end_happy=int(start_pt_ecg+int(marker.loc[happy.index+1,'latency'])*Fs_ECG/Fs_EEG)
    start_anger=int(start_pt_ecg+int(anger['latency'])*Fs_ECG/Fs_EEG)
    end_anger=int(start_pt_ecg+int(marker.loc[anger.index+1,'latency'])*Fs_ECG/Fs_EEG)
    start_disgust=int(start_pt_ecg+int(disgust['latency'])*Fs_ECG/Fs_EEG)
    end_disgust=int(start_pt_ecg+int(marker.loc[disgust.index+1,'latency'])*Fs_ECG/Fs_EEG)
    if start_neutral<0:
        start_neutral=0
    if start_fear<0:
        start_fear=0
    if start_sad<0:
        start_sad=0
    if start_happy<0:
        start_happy=0
    if start_anger<0:
        start_anger=0
    if start_disgust<0:
        start_neutral=0
    n_ECG=ECG_signal[:,start_neutral:end_neutral]
    f_ECG=ECG_signal[:,start_fear:end_fear]
    s_ECG=ECG_signal[:,start_sad:end_sad]
    h_ECG=ECG_signal[:,start_happy:end_happy]
    an_ECG=ECG_signal[:,start_anger:end_anger]
    d_ECG=ECG_signal[:,start_disgust:end_disgust]
    return n_ECG,f_ECG,s_ECG,h_ECG,an_ECG,d_ECG


In [ ]:
files=glob.glob(f"all_data\combined_data\ECG_EX\*.mat")
Experiment_info=pd.read_excel(r"E:\multimodal_signal\all_data\experiment subjects.xlsx",sheet_name='experiment')
for file in files:

    ECG=loadmat(file)
    EEG_start=Experiment_info.loc[Experiment_info.ID=='{}'.format(file[-7:-4])].Date
    EEG=loadmat(r"E:\multimodal_signal\all_data\combined_data\EEG_new1\{}".format(file[-7:]))
    Fs_EEG=EEG['srate']
    marker=pd.DataFrame(EEG['Event_marker'],columns=['event','latency','urevent'])
    ECG_data=ECG['ECG']
    Fs_ECG=ECG['fs']
    start_ECG=ECG['start_time']
    EEG_start.index=np.arange(len(EEG_start))
    end_ECG=ECG['stop_time']
    start=joining(list(start_ECG[0]))
    year,month,day,hour,min,sec= int(start[0:4]),int(start[4:6]),int(start[6:8]),int(start[8:10]),int(start[10:12]),int(start[12:14])
    date=datetime.datetime(year,month,day,hour,min,sec)
    dt=pd.Series(date.strftime('%Y-%m-%d %H:%M:%S'))
    delt=pd.to_datetime(dt)-EEG_start #if neg then problem
    start_pt_ecg=int(delt[0].total_seconds()*Fs_ECG)
    n_ECG,f_ECG,s_ECG,h_ECG,an_ECG,d_ECG=seperate_ECG(start_pt_ecg,Fs_ECG,Fs_EEG,ECG_data,marker)
    ECG_labelled={'neutral':n_ECG,'fear':f_ECG,'sad':s_ECG,'happy':h_ECG,'anger':an_ECG,'disgust':d_ECG}
    savemat(r'E:\multimodal_signal\all_data\combined_data\ECG_labelled\{}'.format(file[-7:]),ECG_labelled)


PPG labelling

In [ ]:

def seperate_E4(start_pt_ecg,Fs_PPG,Fs_EEG,PPG,marker):
    neutral=marker[marker.event==101]
    fear=marker[marker.event==102]
    sad=marker[marker.event==103]
    happy=marker[marker.event==104]
    anger=marker[marker.event==105]
    disgust=marker[marker.event==106]
    start_neutral=int(start_pt_ecg+int(neutral['latency'])*Fs_PPG/Fs_EEG)
    end_neutral=int(start_pt_ecg+int(marker.loc[neutral.index+1,'latency'])*Fs_PPG/Fs_EEG)
    start_fear=int(start_pt_ecg+int(fear['latency'])*Fs_PPG/Fs_EEG)
    end_fear=int(start_pt_ecg+int(marker.loc[fear.index+1,'latency'])*Fs_PPG/Fs_EEG)
    start_sad=int(start_pt_ecg+int(sad['latency'])*Fs_PPG/Fs_EEG)
    end_sad=int(start_pt_ecg+int(marker.loc[sad.index+1,'latency'])*Fs_PPG/Fs_EEG)
    start_happy=int(start_pt_ecg+int(happy['latency'])*Fs_PPG/Fs_EEG)
    end_happy=int(start_pt_ecg+int(marker.loc[happy.index+1,'latency'])*Fs_PPG/Fs_EEG)
    start_anger=int(start_pt_ecg+int(anger['latency'])*Fs_PPG/Fs_EEG)
    end_anger=int(start_pt_ecg+int(marker.loc[anger.index+1,'latency'])*Fs_PPG/Fs_EEG)
    start_disgust=int(start_pt_ecg+int(disgust['latency'])*Fs_PPG/Fs_EEG)
    end_disgust=int(start_pt_ecg+int(marker.loc[disgust.index+1,'latency'])*Fs_PPG/Fs_EEG)
    if start_neutral<0:
        start_neutral=0
    if start_fear<0:
        start_fear=0
    if start_sad<0:
        start_sad=0
    if start_happy<0:
        start_happy=0
    if start_anger<0:
        start_anger=0
    if start_disgust<0:
        start_neutral=0
    n_E4=PPG[start_neutral:end_neutral]
    f_E4=PPG[start_fear:end_fear]
    n_E4=PPG[start_neutral:end_neutral]
    f_E4=PPG[start_fear:end_fear]
    s_E4=PPG[start_sad:end_sad]
    h_E4=PPG[start_happy:end_happy]
    an_E4=PPG[start_anger:end_anger]
    d_E4=PPG[start_disgust:end_disgust]
    return n_E4,f_E4,s_E4,h_E4,an_E4,d_E4

In [ ]:
files=glob.glob(f"all_data\combined_data\E4_data\*.mat")
Experiment_info=pd.read_excel(r"E:\multimodal_signal\all_data\experiment subjects.xlsx",sheet_name='experiment')
for file in files:
    E4=loadmat(file)
    start_time=E4['time']
    PPG=E4['ppg']
    PPG_Fs=E4['ppg_fs']
    Acc=E4['acc']
    Acc_fs=E4['acc_fs']
    eda=E4['eda']
    eda_fs=E4['eda_fs']
    hr=E4['hr']
    hr_fs=E4['hr_fs']
    temp=E4['temp']
    temp_fs=E4['temp_fs']
    EEG_start=Experiment_info.loc[Experiment_info.ID=='{}'.format(file[-7:-4])].Date
    EEG=loadmat(r"E:\multimodal_signal\all_data\combined_data\EEG_new1\{}".format(file[-7:]))
    Fs_EEG=EEG['srate']
    marker=pd.DataFrame(EEG['Event_marker'],columns=['event','latency','urevent'])
    EEG_start.index=np.arange(len(EEG_start))
    delt=pd.to_datetime(start_time)-EEG_start
    start_pt_ecg=int(delt[0].total_seconds()*PPG_Fs)
    start_pt_acc=int(delt[0].total_seconds()*Acc_fs)
    start_pt_eda=int(delt[0].total_seconds()*eda_fs)
    start_pt_hr=int(delt[0].total_seconds()*hr_fs)
    start_pt_temp=int(delt[0].total_seconds()*temp_fs)
    n_PPG,f_PPG,s_PPG,h_PPG,an_PPG,d_PPG=seperate_E4(start_pt_ecg,PPG_Fs,Fs_EEG,PPG,marker)
    n_Acc,f_Acc,s_Acc,h_Acc,an_Acc,d_Acc=seperate_E4(start_pt_acc,Acc_fs,Fs_EEG,Acc,marker)
    n_eda,f_eda,s_eda,h_eda,an_eda,d_eda=seperate_E4(start_pt_eda,eda_fs,Fs_EEG,eda,marker)
    n_hr,f_hr,s_hr,h_hr,an_hr,d_hr=seperate_E4(start_pt_hr,hr_fs,Fs_EEG,hr,marker)
    n_temp,f_temp,s_temp,h_temp,an_temp,d_temp=seperate_E4(start_pt_temp,temp_fs,Fs_EEG,temp,marker)
    labelled={'neutral_PPG':n_PPG,'fear_PPG':f_PPG,'sad_PPG':s_PPG,'happy_PPG':h_PPG,'anger_PPG':an_PPG,'disgust_PPG':d_PPG,'neutral_Acc':n_Acc,'fear_Acc':f_Acc,'sad_Acc':s_Acc,'happy_Acc':h_Acc,'anger_Acc':an_Acc,'disgust_Acc':d_Acc,'neutral_eda':n_eda,'fear_eda':f_eda,'sad_eda':s_eda,'happy_eda':h_eda,'anger_eda':an_eda,'disgust_eda':d_eda,'neutral_hr':n_hr,'fear_hr':f_hr,'sad_hr':s_hr,'happy_hr':h_hr,'anger_hr':an_hr,'disgust_hr':d_hr,'neutral_temp':n_temp,'fear_temp':f_temp,'sad_temp':s_temp,'happy_temp':h_temp,'anger_temp':an_temp,'disgust_temp':d_temp}

    savemat(r'E:\multimodal_signal\all_data\combined_data\E4_labelled\{}'.format(file[-7:]),labelled)

Preprocessing and combining data

In [ ]:
def iir_filter(sig, cutoff, order, btype="bandpass", ftype="butter", fs=512, axis=-1):
    sos = signal.iirfilter(order, cutoff, fs=fs, ftype=ftype, btype=btype, output="sos")
    y = signal.sosfilt(sos, sig, axis=axis)
    return y


def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y
def remove_baseline(x, order=None, method="imodpoly"):
    if method not in ["modpoly", "imodpoly", "zhangfit"]:
        raise Exception("Invalid method. Available methods: `modpoly`, `imodpoly`, `zhangfit`")
    if method != "zhangfit" and not order:
        raise Exception("Order must be provided for `modpoly` and `imodpoly` methods")

    base_obj = BaselineRemoval(x)
    if method == "modpoly":
        return base_obj.ModPoly(order)
    elif method == "imodpoly":
        return base_obj.IModPoly(order)
    elif method == "zhangfit":
        return base_obj.ZhangFit()


def baseline_correction_polynomial(x: np.ndarray, order: int):
    x = (x - x.min()) / (x.max() - x.min())  # Normalize the Signal
    p = np.polyfit(np.arange(len(x)), x, order)
    f = np.poly1d(p)
    baseline = f(np.arange(len(x)))
    y = x - baseline
    x_amp = x.max() - x.min()
    y_amp = y.max() - y.min()
    y = y * (x_amp / y_amp)
    return y


def baseline_correction_als(y, lambda_=10e3, p=0.01, niter=100):
    from scipy import sparse
    from scipy.sparse.linalg import spsolve

    y = (y - y.min()) / (y.max() - y.min())
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    D = lambda_ * D.dot(D.transpose())
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    for i in range(niter):
        W.setdiag(w)
        Z = W + D
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    y = y - z
    return y


def baseline_correction_svd(x, order):
    baseline_values = peakutils.baseline(x)
    x = x - baseline_values
    return x


def moving_average(x, window):
    return np.convolve(x, np.ones(window) / window, mode="same")

def segmentation(x_data,overlap_rate,time_window,seg_data):

    overlap = int((1 - overlap_rate)*time_window)

    for i in range(0,len(x_data),overlap):
        seg_data.append(x_data[i:i+time_window])
    df=pd.DataFrame(seg_data)
    df.dropna(inplace=True)
    return np.array(df)
def bp_filter(x, low_f, high_f, samplerate, plot=False):

    low_cutoff_bp = low_f / (samplerate / 2)
    high_cutoff_bp = high_f / (samplerate / 2)

    [b, a] = signal.butter(5, [low_cutoff_bp, high_cutoff_bp], btype='band')

    x_filt = signal.filtfilt(b, a, x)

    if plot:
        t = np.arange(0, len(x) / samplerate, 1 / samplerate)
        plt.plot(t, x)
        plt.plot(t, x_filt, 'k')
        plt.autoscale(tight=True)
        plt.xlabel('Time')
        plt.ylabel('Amplitude (mV)')
        plt.show()

    return x_filt
def plot_signal(x, samplerate, chname):
    t = np.arange(0, len(x) / samplerate, 1 / samplerate)
    plt.plot(t, x)
    plt.autoscale(tight=True)
    plt.xlabel('Time')
    plt.ylabel('Amplitude (mV)')
    plt.title(chname)
    plt.show()


def notch_filter(x, samplerate, plot=False):
    x = x - np.mean(x)

    high_cutoff_notch = 48 / (samplerate / 2)
    low_cutoff_notch = 51 / (samplerate / 2)

    [b, a] = signal.butter(4, [high_cutoff_notch, low_cutoff_notch], btype='stop')

    x_filt = signal.filtfilt(b, a, x.T)

    if plot:
        t = np.arange(0, len(x) / samplerate, 1 / samplerate)
        plt.plot(t, x)
        plt.plot(t, x_filt.T, 'k')
        plt.autoscale(tight=True)
        plt.xlabel('Time')
        plt.ylabel('Amplitude (mV)')
        plt.show()

    return x_filt
def segmentation_baseline_correction(x_data,overlap_rate,time_window,seg_data,baseline_order):

    overlap = int((1 - overlap_rate)*time_window)

    #segment and keep the labels
    for i in range(0,len(x_data),overlap):
        seg_data.append(baseline_correction_polynomial(x_data[i:i+time_window],order=baseline_order))
        #y_segmented_list.append(x_data['activity'][i])
    df=pd.DataFrame(seg_data)
    df.dropna(inplace=True)
    return df
def resample_signal(sig,initial_fs,target_fs):
    resampled_size = int((len(sig) * target_fs) / initial_fs)
    resampled = signal.resample(sig, resampled_size)
    return resampled

filter and segment data

In [ ]:
#functions
def PPG_preprocess(PPG,PPG_lf,PPG_hf,PPG_fs,order):
    PPG=np.array([i[0] for i in PPG])
    PPG_f=bp_filter(PPG,PPG_lf,PPG_hf,PPG_fs)
    PPG_baseline_removed=baseline_correction_polynomial(PPG_f,order)
    return PPG_baseline_removed

def Acc_preprocess(Acc,Acc_lf,Acc_hf,Acc_fs,order):
    channels=[]
    for i in range(Acc.shape[1]):
        ch=Acc[:,i]
        Acc_f=bp_filter(ch,Acc_lf,Acc_hf,Acc_fs)
        Acc_baseline_removed=baseline_correction_polynomial(Acc_f,order)
        channels.append(Acc_baseline_removed)
    return pd.DataFrame(channels).T

def EEG_preprocess(EEG,EEG_lf,EEG_hf,EEG_fs,order):
    channels=[]
    for i in range(EEG.shape[1]):
        ch=EEG[:,i]
        baseline_corrected=baseline_correction_polynomial(ch,order)
        x_n=notch_filter(baseline_corrected,EEG_fs,plot=False)  # to filter out power line noise with minimal disruption to the rest of the signal.
        filtered=butter_lowpass_filter(x_n, EEG_hf, EEG_fs,order=order)
        channels.append(filtered)
    return pd.DataFrame(channels).T
def ECG_preprocess(ECG,ECG_lf,ECG_hf,ECG_fs,order):
    ECG=np.array([i[0] for i in ECG])
    ECG_f=bp_filter(ECG,ECG_lf,ECG_hf,ECG_fs)
    ECG_n=notch_filter(ECG_f,ECG_fs,plot=False)
    ECG_baseline_removed=baseline_correction_polynomial(ECG_n,order)
    avg=moving_average(ECG_baseline_removed,10)
    return avg
def segmentation_channel(x_data,overlap_rate,time_window,ch_list):
    for channel in range(x_data.shape[1]):
        ch=x_data[:,channel]
        #ch_list.append(segmentation(ch,overlap_rate,time_window,seg_data=[]))
        ch_list['ch_{}'.format(channel+1)]=segmentation(ch,overlap_rate,time_window,seg_data=[])

    return ch_list

In [ ]:
import numpy as np
files=glob.glob(f"all_data\combined_data\E4_labelled\*.mat")
target_fs=128
EEG_fs=250
ECG_fs=512
PPG_fs=64
Acc_fs=32
Eda_fs=4
temp_fs=4
hr_fs=1
PPG_hf=10
PPG_lf=0.5
Acc_lf=0.05
Acc_hf=10
EEG_lf,EEG_hf=(0.2,70)
ECG_lf,ECG_hf=(0.2,60)
baseline_order=6
overlap_rate=0
time_win=1024

for file in files:
    E4=loadmat(file)
    EEG_file=loadmat(r'E:\multimodal_signal\all_data\combined_data\EEG_labeled\{}.mat'.format(file[-7:-4]))
    ECG_file=loadmat(r'E:\multimodal_signal\all_data\combined_data\ECG_labelled\{}.mat'.format(file[-7:-4]))
    emotions={}
    labels=['neutral','fear','anger','sad','happy','disgust']
    for label in labels:
        PPG=E4['{}_PPG'.format(label)]
        Acc=E4['{}_Acc'.format(label)]
        Temp=E4['{}_temp'.format(label)]
        Eda=E4['{}_eda'.format(label)]
        hr=E4['{}_hr'.format(label)]
        hr=np.array([i[0] for i in hr])
        Eda=np.array([i[0] for i in Eda])
        Temp=np.array([i[0] for i in Temp])
        EEG=EEG_file[label].T
        ECG=ECG_file[label].T
        try:
            #filter and baseline removal
            PPG_filt=PPG_preprocess(PPG,PPG_lf,PPG_hf,PPG_fs,baseline_order)
            Acc_filt=Acc_preprocess(Acc,Acc_lf,Acc_hf,Acc_fs,baseline_order)
            EEG_filt=EEG_preprocess(EEG,EEG_lf,EEG_hf,EEG_fs,baseline_order)
            ECG_filt=ECG_preprocess(ECG,ECG_lf,ECG_hf,ECG_fs,baseline_order)
            #resample
            resampled_EEG=resample_signal(EEG_filt,EEG_fs,target_fs)
            resampled_ECG=resample_signal(ECG_filt,ECG_fs,target_fs)
            resampled_PPG=resample_signal(PPG_filt,PPG_fs,target_fs)
            resampled_Acc=resample_signal(Acc_filt,Acc_fs,target_fs)
            resampled_eda=resample_signal(Eda,Eda_fs,target_fs)
            #resampled_hr=resample_signal(hr,hr_fs,target_fs)
            resampled_temp=resample_signal(Temp,temp_fs,target_fs)
            #segmentation
            seg_PPG=segmentation(resampled_PPG,overlap_rate,time_win,seg_data=[])
            seg_EEG=segmentation_channel(resampled_EEG,overlap_rate,time_win,ch_list={})
            seg_Acc=segmentation_channel(resampled_Acc,overlap_rate,time_win,ch_list={})
            seg_ECG=segmentation(resampled_ECG,overlap_rate,time_win,seg_data=[])
            seg_Eda=segmentation(resampled_eda,overlap_rate,time_win,seg_data=[])
            seg_Temp=segmentation(resampled_temp,overlap_rate,time_win,seg_data=[])
            #seg_hr=segmentation(resampled_hr,overlap_rate,time_win,seg_data=[])
            emotions['{}'.format(label)]={'ch1_EEG':seg_EEG['ch_1'],'ch2_EEG':seg_EEG['ch_2'],'ch3_EEG':seg_EEG['ch_3'],'ch4_EEG':seg_EEG['ch_4'],'ch5_EEG':seg_EEG['ch_5'],'ch6_EEG':seg_EEG['ch_6'],'ch7_EEG':seg_EEG['ch_7'],'ch8_EEG':seg_EEG['ch_8'],'ch9_EEG':seg_EEG['ch_9'],'ch10_EEG':seg_EEG['ch_10'],'PPG':seg_PPG,'ch1_Acc':seg_Acc['ch_1'],'ch2_Acc':seg_Acc['ch_2'],'ch3_Acc':seg_Acc['ch_3'],'ECG':seg_ECG,'EDA':seg_Eda,'Temp':seg_Temp}
            if len(emotions) == 0:
                print(file)
        except:pass

    #if len(emotions)!= 0:
        #savemat(r"E:\multimodal_signal\all_data\combined_data\multimodal_segmented_data\{}".format(file[-7:]),emotions)

In [ ]:
from sklearn import preprocessing
#preprocessing.normalize(([seg_EEG['ch_2'][0,:]]))

plot_signal(EEG[:,1][:2000],250,'before_preprocess')
plot_signal(seg_EEG['ch_2'][0,:],128,'after preprocess')

In [ ]:
plot_signal(ECG[:4096],512,'before_preprocess')
plot_signal(seg_ECG[0,:],128,'after preprocess')

In [ ]:
plot_signal(PPG[:510],64,'before_preprocess')
plot_signal(seg_PPG[0,:],128,'after preprocess')

In [ ]:
plot_signal(Eda[:32],4,'before_preprocess')
plot_signal(seg_Eda[0,:],128,'after preprocess')

In [ ]:
types=6

Fs=128
emotions=[]
segmented=[]
labels=['neutral', 'fear', 'anger', 'sad', 'happy', 'disgust']
dict_labels={'neutral':0, 'fear':1, 'anger':1, 'sad':1, 'happy':2, 'disgust':1}
files=glob.glob(r'E:\multimodal_signal\all_data\combined_data\multimodal_segmented_data\*')
for filename in files:
  print(filename)
  mat=loadmat(filename)
  for label in labels:
    try:
       PPG=pd.DataFrame(mat[label]['PPG'][0,0])
       PPG['label']=dict_labels[label]
       PPG['id']=filename[-7:-4]
       segmented.append(PPG)
    except:pass

In [ ]:
ch1_df=pd.concat([segmented[i] for i in range(len(segmented))], axis=0,ignore_index=True)
X=np.array(ch1_df.drop(['label','id'],axis=1))
Y=np.array(ch1_df.label)
Y = Y.reshape((X.shape[0],1))
id=np.array(ch1_df.id).reshape(X.shape[0],1)
pd.DataFrame(Y).value_counts()

In [ ]:
savemat('PPG_data_for_feature_extract.mat',{'ppg_data':X,'label':Y,'id':id})